To Do:
- Deal with records that have RDATE before ADATE
- (?) Do something with this information (from metadata text file):
                            LL mailings had labels only
                            WL mailings had labels only
                            CC mailings are calendars with stickers but do
                               not have labels
                            FS mailings are blank cards that fold into
                               thirds with labels
                            NK mailings are blank cards with labels
                            SK mailings are blank cards with labels
                            TK mailings have thank you printed on the
                               outside with labels
                            GK mailings are general greeting cards (an
                               assortment of birthday, sympathy, blank, & get
                               well) with labels
                            XK mailings are Christmas cards with labels
                            X1 mailings have labels and a notepad
                            G1 mailings have labels and a notepad
- Use the variable that is most correlated with 'Age' ('Age' is yet to be created from 'DOB') to fill in the missing values of 'Age' (using a linear model for example)

In [2]:
# Import libaries
import pandas as pd
import os
import numpy as np

In [3]:
# Load data
donors = pd.read_csv(os.path.join('Data', 'donors.csv'), sep=',', index_col=0)

/Users/philippmetzger/opt/anaconda3/envs/data_mining/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
donors

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,7.741935,95515,0,L,4,E,X,X,X,C
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,7.481481,15078,1,L,4,E,X,X,X,C
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,6.812500,172556,1,L,4,E,X,X,X,C
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,6.864865,7112,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,ASE,1,AK,99504,,,NaN,0,,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,2016-01-01,DCD,1,TX,77379,,,1970-01-01,0,,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,2015-01-01,MBC,1,MI,48910,,,1958-01-01,0,,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,2006-01-01,PRV,0,CA,91320,,,1960-05-01,0,X,...,12.146341,4693,1,L,4,F,X,X,X,A


In [5]:
list(donors)

['ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'INCOME',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',


### Treat the cases where 'LASTDATE' suggests that donor is not lapsing or lapsed (In this dataset, all donors should be either lapsing or lapsed)

My interpretation of forum entry: "Lapsed donors": 
Observations that have a value for 'LASTDATE' that is more recent than 13 months 
before the later date of the second promotion being sent off (2017-06-01), they are not lapsing or lapsed and thus 
inconsistencies.

In [6]:
# Get the dates when the most recent campaign was mailed
pd.unique(donors['ADATE_2'])

array(['2017-06-01', '2017-04-01'], dtype=object)

In [7]:
# Get unique values of 'LASTDATE'
np.sort((pd.unique(donors['LASTDATE'])))

array(['2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01',
       '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01',
       '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01',
       '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01',
       '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01',
       '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01'],
      dtype=object)

In [8]:
# Remove inconsistent values from donors
donors_2 = donors[~donors['LASTDATE'].isin(['2016-06-01','2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01','2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01'])

In [14]:
# Accept the changes made
donors = donors_2

In [15]:
# List the variables that have missing values and their missing value counts
donors.isna().sum()[donors.isna().sum()!=0]

DOB         22883
NUMCHLD     79375
INCOME      20461
WEALTH1     43052
MBCRAFT     50858
            ...  
RAMNT_23    84033
RAMNT_24    74596
NEXTDATE     9973
TIMELAG      9973
GEOCODE2      127
Length: 91, dtype: int64

In [70]:
# Get the total number of nan values
donors.isna().sum()[donors.isna().sum()!=0].sum()

4958912

In [17]:
# See how many duplicated observations exist
donors.duplicated().sum()

0

In [71]:
# Replace " " by nans
donors.replace(" ", np.nan, inplace=True)

/Users/philippmetzger/opt/anaconda3/envs/data_mining/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [72]:
# Get the total number of nan values once more
donors.isna().sum()[donors.isna().sum()!=0].sum()

8394885

In [73]:
# Replace "" by nans
donors.replace("", np.nan, inplace=True)

/Users/philippmetzger/opt/anaconda3/envs/data_mining/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [75]:
# Get the total number of nan values once more
donors.isna().sum()[donors.isna().sum()!=0].sum()

8394885

In [76]:
# Create some descriptive statistics
desc_stats = donors.describe(include="all")  # try with all and without all

# View a certain one
desc_stats['TCODE']

count     91447.000000
unique             NaN
top                NaN
freq               NaN
mean         53.474122
std         943.088492
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max       72002.000000
Name: TCODE, dtype: float64

In [77]:
# Get all data types and save them in 'dtypes'
dtypes = donors.dtypes

In [79]:
# Define a function to convert numerical String features to float
def to_num(df, colname):
    """This function takes a dataframe and a column name and converts the column with this name to float"""
    
    # column = df[colname]
    # col_index = df.columns.get_loc(colname)
    
    df[colname] = pd.to_numeric(df[colname],errors='coerce')

In [80]:
# Test 'to_num' on variable 'SOLP3'
to_num(donors, 'SOLP3')

<ipython-input-79-8b67fbe963ee>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname] = pd.to_numeric(df[colname],errors='coerce')


In [139]:
## Obtain information of the type of a certain feature
feature = 'DOB'

# Print its type
print('dtype of feature', feature, ':', dtypes[feature])

# Take a closer look at the first non nan element of it
first_non_na_element = donors[feature][~donors[feature].isna()].iloc[0]
print('First non nan element of this feature:', first_non_na_element)
print('Class of the first element of this feature:', type(first_non_na_element))

# View the unique values of this feature
# print('Sorted unique values of feature', feature, ':', np.sort(pd.unique(donors[feature])))
print('Unique values of feature', feature, ':', pd.unique(donors[feature]))

dtype of feature DOB : object
First non nan element of this feature: 1957-12-01
Class of the first element of this feature: <class 'str'>
Unique values of feature DOB : ['1957-12-01' '1972-02-01' nan '1948-01-01' '1940-01-01' '1980-01-01'
 '1952-11-01' '1943-01-01' '1946-03-01' '1947-09-01' '1974-01-01'
 '1972-01-01' '1956-01-01' '1936-01-01' '1943-11-01' '1963-07-01'
 '1976-01-01' '1934-01-01' '1968-09-01' '1946-01-01' '1949-04-01'
 '1949-01-01' '1930-02-01' '1933-11-01' '1988-01-01' '1973-10-01'
 '1951-10-01' '2010-08-01' '1957-06-01' '1950-01-01' '1972-10-01'
 '1957-03-01' '1978-01-01' '1970-01-01' '1984-01-01' '1938-01-01'
 '1982-01-01' '1968-01-01' '1977-01-01' '1946-08-01' '1964-03-01'
 '1945-12-01' '1948-07-01' '1956-05-01' '1971-01-01' '1964-01-01'
 '1952-01-01' '1985-01-01' '1955-01-01' '1954-01-01' '1958-12-01'
 '1941-01-01' '1944-10-01' '1931-01-01' '1967-07-01' '2000-09-01'
 '1934-08-01' '1949-09-01' '1959-01-01' '1940-06-01' '1932-06-01'
 '1966-01-01' '1966-12-01' '1957-01

In [124]:
# Create a list with the metric features
# Including dates and other variables that might still be Strings at this point
# Will use 'to_num' on all metric features in the next step
metric_features = ['ODATEDW',
 'DOB',
 'NUMCHLD',
 'INCOME',
 'WEALTH1',
 'HIT',
                   # Above: Information about the donor
                   # Below: About the number of times the donor has responded to other types of mail order offers
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
                   # Below: Data from third party regarding the household or neighborhood (According to David Silva) 
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',         
 'SOLP3',
 'SOLIH',
 'WEALTH2',
                   # Below: About donor's neighbourhood
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',              
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',           
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU1',
 'HU2',
 'HU3',
 'HU4',
 'HU5',                  
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP1',
 'HVP2',
 'HVP3',
 'HVP4',
 'HVP5',
 'HVP6',                  
 'HUR1',
 'HUR2',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA2',
 'HUPA3',
 'HUPA4',
 'HUPA5',
 'HUPA6',
 'HUPA7',
 'RP1',
 'RP2',
 'RP3',
 'RP4',                 
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC6',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC11',
 'IC12',
 'IC13',
 'IC14',
 'IC15',
 'IC16',
 'IC17',
 'IC18',
 'IC19',
 'IC20',
 'IC21',
 'IC22',
 'IC23',           
 'HHAS1',
 'HHAS2',
 'HHAS3',
 'HHAS4',
 'MC1',
 'MC2',
 'MC3',
 'TPE1',
 'TPE2',
 'TPE3',
 'TPE4',
 'TPE5',
 'TPE6',
 'TPE7',
 'TPE8',
 'TPE9',
 'PEC1',
 'PEC2',
 'TPE10',
 'TPE11',
 'TPE12',
 'TPE13',
 'LFC1',
 'LFC2',
 'LFC3',
 'LFC4',
 'LFC5',
 'LFC6',
 'LFC7',
 'LFC8',
 'LFC9',
 'LFC10',
 'OCC1',
 'OCC2',
 'OCC3',
 'OCC4',
 'OCC5',
 'OCC6',
 'OCC7',
 'OCC8',
 'OCC9',
 'OCC10',
 'OCC11',
 'OCC12',
 'OCC13',
 'EIC1',
 'EIC2',
 'EIC3',
 'EIC4',
 'EIC5',
 'EIC6',
 'EIC7',
 'EIC8',
 'EIC9',
 'EIC10',
 'EIC11',
 'EIC12',
 'EIC13',
 'EIC14',
 'EIC15',
 'EIC16',
 'OEDC1',
 'OEDC2',
 'OEDC3',
 'OEDC4',
 'OEDC5',
 'OEDC6',
 'OEDC7',
 'EC1',
 'EC2',
 'EC3',
 'EC4',
 'EC5',
 'EC6',
 'EC7',
 'EC8',
 'SEC1',
 'SEC2',
 'SEC3',
 'SEC4',
 'SEC5',
 'AFC1',
 'AFC2',
 'AFC3',
 'AFC4',
 'AFC5',
 'AFC6',
 'VC1',
 'VC2',
 'VC3',
 'VC4',
 'ANC1',
 'ANC2',
 'ANC3',
 'ANC4',
 'ANC5',
 'ANC6',
 'ANC7',
 'ANC8',
 'ANC9',
 'ANC10',
 'ANC11',
 'ANC12',
 'ANC13',
 'ANC14',
 'ANC15',
 'POBC1',
 'POBC2',
 'LSC1',
 'LSC2',
 'LSC3',
 'LSC4',
 'VOC1',
 'VOC2',
 'VOC3',
 'HC1',
 'HC2',
 'HC3',
 'HC4',
 'HC5',
 'HC6',
 'HC7',
 'HC8',
 'HC9',
 'HC10',
 'HC11',
 'HC12',
 'HC13',
 'HC14',
 'HC15',
 'HC16',
 'HC17',
 'HC18',
 'HC19',
 'HC20',
 'HC21',
 'MHUC1',
 'MHUC2',
 'AC1',
 'AC2',
                   # Above: About donor's neighbourhood      
                   # Below: Date promotion X was mailed
 'ADATE_2',
 'ADATE_3',
 'ADATE_4',
 'ADATE_5',
 'ADATE_6',
 'ADATE_7',
 'ADATE_8',
 'ADATE_9',
 'ADATE_10',
 'ADATE_11',
 'ADATE_12',
 'ADATE_13',
 'ADATE_14',
 'ADATE_15',
 'ADATE_16',
 'ADATE_17',
 'ADATE_18',
 'ADATE_19',
 'ADATE_20',
 'ADATE_21',
 'ADATE_22',
 'ADATE_23',
 'ADATE_24',
                   # Below: Information about how many promotions donor has received
 'CARDPROM',
 'MAXADATE',
 'NUMPROM',
 'CARDPM12',
 'NUMPRM12',
                   # Below: Date the donation was received
 'RDATE_3',
 'RDATE_4',
 'RDATE_5',
 'RDATE_6',
 'RDATE_7',
 'RDATE_8',
 'RDATE_9',
 'RDATE_10',
 'RDATE_11',
 'RDATE_12',
 'RDATE_13',
 'RDATE_14',
 'RDATE_15',
 'RDATE_16',
 'RDATE_17',
 'RDATE_18',
 'RDATE_19',
 'RDATE_20',
 'RDATE_21',
 'RDATE_22',
 'RDATE_23',
 'RDATE_24',
                   # Below: Dollar amount of the donation
 'RAMNT_3',
 'RAMNT_4',
 'RAMNT_5',
 'RAMNT_6',
 'RAMNT_7',
 'RAMNT_8',
 'RAMNT_9',
 'RAMNT_10',
 'RAMNT_11',
 'RAMNT_12',
 'RAMNT_13',
 'RAMNT_14',
 'RAMNT_15',
 'RAMNT_16',
 'RAMNT_17',
 'RAMNT_18',
 'RAMNT_19',
 'RAMNT_20',
 'RAMNT_21',
 'RAMNT_22',
 'RAMNT_23',
 'RAMNT_24',
                   # Below: Summary variables for this donor
 'RAMNTALL',
 'NGIFTALL',
 'CARDGIFT',
 'MINRAMNT',
 'MINRDATE',
 'MAXRAMNT',
 'MAXRDATE',
 'LASTGIFT',
 'LASTDATE',
 'FISTDATE',
 'NEXTDATE',
 'TIMELAG',
 'AVGGIFT']

print('Number of metric features:', len(metric_features))

Number of metric features: 398


In [125]:
# Create a list with the non metrics features by excluding the metric ones
non_metric_features = donors.columns.drop(metric_features).to_list()

print('Number of non-metric features:', len(non_metric_features))

Number of non-metric features: 77


In [142]:
# List the variables that have missing values and their missing value counts
nan_features = donors.isna().sum()[donors.isna().sum()!=0]
nan_features

OSOURCE       665
MAILCODE    90106
PVASTATE    90810
DOB         22883
NOEXCH          6
            ...  
RAMNT_23    84033
RAMNT_24    74596
NEXTDATE     9973
TIMELAG      9973
GEOCODE2      309
Length: 153, dtype: int64

In [160]:
# Get the percentage of nan values in a feature
feature = 'ADATE_6'
print('Percentage of nan values for', feature, ':', round(len(donors[feature][donors[feature].isna()]) / len(donors[feature]) * 100, 6))

Percentage of nan values for ADATE_6 : 3.728936
